In [1]:
import sys

import jax as jax

sys.path.append('..//')

import jax
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import jit
import time
import warnings

jax.config.update("jax_enable_x64", False)
jax.config.update("jax_platform_name", "cpu")

from parsmooth._base import MVNStandard, FunctionalModel, MVNSqrt
from parsmooth.linearization import cubature, extended
from parsmooth.methods import iterated_smoothing
from bearing_data import get_data, make_parameters

In [2]:
warnings.simplefilter("ignore")

In [3]:
s1 = jnp.array([-1.5, 0.5])  # First sensor location
s2 = jnp.array([1., 1.])  # Second sensor location
r = 0.5  # Observation noise (stddev)
x0 = jnp.array([0.1, 0.2, 1, 0])  # initial true location
dt = 0.01  # discretization time step
qc = 0.01  # discretization noise
qw = 0.1  # discretization noise

T = 5000
_, true_states, ys = get_data(x0, dt, r, T, s1, s2)
Q, R, observation_function, transition_function = make_parameters(qc, qw, r, dt, s1, s2)

chol_Q = jnp.linalg.cholesky(Q)
chol_R = jnp.linalg.cholesky(R)

m0 = jnp.array([-4., -1., 2., 7., 3.])
chol_P0 = jnp.eye(5)
P0 = jnp.eye(5)

init = MVNStandard(m0, P0)
chol_init = MVNSqrt(m0, chol_P0)

initial_states =  MVNStandard(jnp.repeat(jnp.array([[-1., -1., 6., 4., 2.]]),T + 1, axis=0),
                                                     jnp.repeat(jnp.eye(5).reshape(1, 5, 5), T + 1, axis=0))
initial_states_sqrt = MVNSqrt(jnp.repeat(jnp.array([[-1., -1., 6., 4., 2.]]),T + 1, axis=0),
                              jnp.repeat(jnp.eye(5).reshape(1, 5, 5), T + 1, axis=0))


sqrt_transition_model = FunctionalModel(transition_function, MVNSqrt(jnp.zeros((5,)), chol_Q))
transition_model = FunctionalModel(transition_function, MVNStandard(jnp.zeros((5,)), Q))

sqrt_observation_model = FunctionalModel(observation_function, MVNSqrt(jnp.zeros((2,)), chol_R))
observation_model = FunctionalModel(observation_function, MVNStandard(jnp.zeros((2,)), R))

# Cubature

In [4]:
def cubature_sqrt_seq(y, initial_points_sqrt, iteration):
    sqrt_seq_res = iterated_smoothing(y, chol_init, sqrt_transition_model, sqrt_observation_model,
                                      cubature, initial_points_sqrt, False,
                                      criterion=lambda i, *_: i < iteration)
    return sqrt_seq_res


def cubature_sqrt_par(y, initial_points_sqrt, iteration):
    sqrt_par_res = iterated_smoothing(y, chol_init, sqrt_transition_model, sqrt_observation_model,
                                      cubature, initial_points_sqrt, True,
                                      criterion=lambda i, *_: i < iteration)
    return sqrt_par_res


def cubature_std_seq(y, initial_points, iteration):
    std_seq_res = iterated_smoothing(y, init, transition_model, observation_model,
                                     cubature, initial_points, False,
                                     criterion=lambda i, *_: i < iteration)
    return std_seq_res


def cubature_std_par(y, initial_points, iteration):
    std_par_res = iterated_smoothing(y, init, transition_model, observation_model,
                                     cubature, initial_points, True,
                                     criterion=lambda i, *_: i < iteration)
    return std_par_res

In [5]:
def func(method, lengths, n_runs=100, n_iter=10, sqrt=False):
    res_mean = []
    res_median = []
    for k, j in enumerate(lengths):
        print(f"Iteration {k+1} out of {len(lengths)}", end="\r")
        observations_slice = ys[:j]
        
        if sqrt:
            init_linearizations_points_slice = initial_states_sqrt.mean[:j+1]
            init_linearizations_chols_slice = initial_states_sqrt.chol[:j+1]
            init_linearizations_states = MVNSqrt(init_linearizations_points_slice, init_linearizations_chols_slice)
            args = observations_slice, init_linearizations_states, n_iter
        else:
            init_linearizations_points_slice = initial_states.mean[:j+1]
            init_linearizations_covs_slice = initial_states.cov[:j+1]
            init_linearizations_states = MVNStandard(init_linearizations_points_slice, init_linearizations_covs_slice)
            args = observations_slice, init_linearizations_states, n_iter
            
        s = method(*args)                
        s.mean.block_until_ready()
        run_times = []
        for _ in range(n_runs):
            tic = time.time()
            s_states = method(*args)
            s_states.mean.block_until_ready()
            toc = time.time()
            run_times.append(toc - tic)
        res_mean.append(np.mean(run_times))
        res_median.append(np.median(run_times))
    print()
    return np.array(res_mean)#, np.array(res_median)

In [6]:
lengths_space = np.logspace(2, int(np.log2(T)), num=10, base=2, dtype=int)


In [7]:
cpu_cubature_sqrt_par = jit(cubature_sqrt_par, backend="cpu")
cpu_cubature_sqrt_seq = jit(cubature_sqrt_seq, backend="cpu")
cpu_cubature_std_seq = jit(cubature_std_seq, backend="cpu")
cpu_cubature_std_par = jit(cubature_std_par, backend="cpu")


In [ ]:
warnings.filterwarnings("ignore")

cpu_cubature_sqrt_par_mean_time, cpu_cubature_sqrt_par_median_time = func(cpu_cubature_sqrt_par, lengths_space, sqrt=True)



2021-11-18 10:25:52.185314: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:55] 
********************************
Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
Compiling module jit_cubature_sqrt_par.64867
********************************


2021-11-18 10:29:11.085499: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:55] 
********************************
Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
Compiling module jit_cubature_sqrt_par__1.85607
********************************


2021-11-18 10:35:44.144093: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:55] 
********************************
Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
Compiling module jit_cubature_sqrt_par__2.106331
********************************


In [ ]:
warnings.filterwarnings("ignore")
cpu_cubature_sqrt_seq_mean_time, cpu_cubature_sqrt_seq_median_time = func(cpu_cubature_sqrt_seq, lengths_space, sqrt = True)



In [ ]:
warnings.filterwarnings("ignore")
cpu_cubature_std_seq_mean_time, cpu_cubature_std_seq_median_time = func(cpu_cubature_std_seq, lengths_space, sqrt = False)


In [ ]:
warnings.filterwarnings("ignore")
cpu_cubature_std_par_mean_time, cpu_cubature_std_par_median_time = func(cpu_cubature_std_par, lengths_space, sqrt = False)


In [ ]:
jnp.savez("CPU-cubature-newtria-T5e3-100runs-mean-median-time",
          cpu_cubature_std_seq_mean_time=cpu_cubature_std_seq_mean_time,
          cpu_cubature_std_seq_median_time=cpu_cubature_std_seq_median_time,
          
          cpu_cubature_std_par_mean_time=cpu_cubature_std_par_mean_time,
          cpu_cubature_std_par_median_time=cpu_cubature_std_par_median_time,
          
          cpu_cubature_sqrt_seq_mean_time=cpu_cubature_sqrt_seq_mean_time,
          cpu_cubature_sqrt_seq_median_time=cpu_cubature_sqrt_seq_median_time,
          
          cpu_cubature_sqrt_par_mean_time=cpu_cubature_sqrt_par_mean_time,
          cpu_cubature_sqrt_par_median_time=cpu_cubature_sqrt_par_median_time)


In [ ]:
plt.figure(figsize=(10,7))
plt.loglog(lengths_space, cpu_cubature_std_seq_mean_time, label="cubature-std-Seq-CPU", linestyle="-.", linewidth=3)
plt.loglog(lengths_space, cpu_cubature_std_par_mean_time, label="cubature-std-Par-CPU", linestyle="-.", linewidth=3)
plt.loglog(lengths_space, cpu_cubature_sqrt_par_mean_time, label="cubature-sqrt-Par-CPU", linestyle="-.", linewidth=3)
plt.loglog(lengths_space, cpu_cubature_sqrt_seq_mean_time, label="cubature-sqrt-Seq-CPU", linestyle="-.", linewidth=3)


plt.grid(True, which="both")
plt.title("cubature-CPU, sequential v.s parallel, std v.s sqrt")
plt.legend()
plt.show()

# Extended

In [ ]:
def extended_sqrt_seq(y, initial_points_sqrt, iteration):
    sqrt_seq_res = iterated_smoothing(y, chol_init, sqrt_transition_model, sqrt_observation_model,
                                      extended, initial_points_sqrt, False,
                                      criterion=lambda i, *_: i < iteration)
    return sqrt_seq_res


def extended_sqrt_par(y, initial_points_sqrt, iteration):
    sqrt_par_res = iterated_smoothing(y, chol_init, sqrt_transition_model, sqrt_observation_model,
                                      extended, initial_points_sqrt, True,
                                      criterion=lambda i, *_: i < iteration)
    return sqrt_par_res


def extended_std_seq(y, initial_points, iteration):
    std_seq_res = iterated_smoothing(y, init, transition_model, observation_model,
                                     extended, initial_points, False,
                                     criterion=lambda i, *_: i < iteration)
    return std_seq_res


def extended_std_par(y, initial_points, iteration):
    std_par_res = iterated_smoothing(y, init, transition_model, observation_model,
                                     extended, initial_points, True,
                                     criterion=lambda i, *_: i < iteration)
    return std_par_res

In [ ]:
cpu_extended_sqrt_par = jit(extended_sqrt_par, backend="cpu")
cpu_extended_sqrt_seq = jit(extended_sqrt_seq, backend="cpu")
cpu_extended_std_seq = jit(extended_std_seq, backend="cpu")
cpu_extended_std_par = jit(extended_std_par, backend="cpu")

In [ ]:
warnings.filterwarnings("ignore")
cpu_extended_sqrt_par_mean_time, cpu_extended_sqrt_par_median_time = func(cpu_extended_sqrt_par, lengths_space, sqrt=True)


In [ ]:
warnings.filterwarnings("ignore")
cpu_extended_sqrt_seq_mean_time, cpu_extended_sqrt_seq_median_time = func(cpu_extended_sqrt_seq, lengths_space, sqrt=True)


In [ ]:
warnings.filterwarnings("ignore")
cpu_extended_std_seq_mean_time, cpu_extended_std_seq_median_time = func(cpu_extended_std_seq, lengths_space, sqrt=False)


In [ ]:
warnings.filterwarnings("ignore")
cpu_extended_std_par_mean_time, cpu_extended_std_par_median_time = func(cpu_extended_std_par, lengths_space, sqrt=False)


In [ ]:
jnp.savez("CPU-extended-newtria-T5e3-100runs-mean-mediantime",
          cpu_extended_std_seq_mean_time=cpu_extended_std_seq_mean_time,
          cpu_extended_std_seq_median_time=cpu_extended_std_seq_median_time,
          
          cpu_extended_std_par_mean_time=cpu_extended_std_par_mean_time,
          cpu_extended_std_par_median_time=cpu_extended_std_par_median_time,
          
          cpu_extended_sqrt_seq_mean_time=cpu_extended_sqrt_seq_mean_time,
          cpu_extended_sqrt_seq_median_time=cpu_extended_sqrt_seq_median_time,
          
          cpu_extended_sqrt_par_mean_time=cpu_extended_sqrt_par_mean_time,
          cpu_extended_sqrt_par_median_time=cpu_extended_sqrt_par_median_time)